In [2]:
import pandas as pd
import numpy as np
from IPython.display import display

#import original data folder
data_folder = "C:/Users/roman/OneDrive/Bureau/UM/BA/Y3/P1/RS/Group Assignment/archive"

#preparing movies
movies = pd.read_csv(data_folder + "/movie.csv")
movies_df = movies.rename(columns={'movieId': 'movie_id', 'title': 'title', 'genres': 'genres'})
movies_df = movies_df[['movie_id', 'title', 'genres']]
display(movies_df)

#preparing ratings
ratings = pd.read_csv(data_folder + "/rating.csv")
ratings_df = ratings.rename(columns={'userId': 'user_id', 'movieId': 'movie_id', 'rating': 'rating', 'timestamp': 'timestamp'})
ratings_df = ratings_df[['user_id', 'movie_id', 'rating']]
display(ratings_df)

#preparing movies_tags
genome_tags = pd.read_csv(data_folder + "/genome_tags.csv")
genome_tags_df = genome_tags.rename(columns={'tagId': 'tag_id', 'tag': 'tag'})
movies_tags = pd.read_csv(data_folder + "/genome_scores.csv")
movies_tags_df = movies_tags.rename(columns={'movieId': 'movie_id', 'tagId': 'tag_id', 'relevance': 'relevance'})

movies_tags_df.insert(2, "tag", genome_tags_df["tag"])
display(movies_tags_df)


,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


,user_id,movie_id,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
...,...,...,...
20000258,138493,68954,4.5
20000259,138493,69526,4.5
20000260,138493,69644,3.0
20000261,138493,70286,5.0


,movie_id,tag_id,tag,relevance
0,1,1,007,0.02500
1,1,2,007 (series),0.02500
2,1,3,18th century,0.05775
3,1,4,1920s,0.09675
4,1,5,1930s,0.14675
...,...,...,...,...
11709763,131170,1124,NaN,0.58775
11709764,131170,1125,NaN,0.01075
11709765,131170,1126,NaN,0.01575
11709766,131170,1127,NaN,0.11450


In [25]:
movies_df = movies_df.drop_duplicates(subset='title', keep='first')

#join ratings to genres
ratings_genres_df = pd.merge(movies_df, ratings_df,  how='inner', left_on=['movie_id'], right_on = ['movie_id'])
ratings_genres_df = ratings_genres_df[['movie_id', 'user_id', 'rating', 'title', 'genres']]

#keep only users and genres
users_genres = ratings_genres_df.loc[:, ['user_id', 'genres']]

#new users only dataframe
users_df = users_genres.drop_duplicates(subset=['user_id'], keep='first')
users_df = users_df['user_id']
users_df = pd.DataFrame(users_df)
users_df = users_df.set_index('user_id')
users_df = users_df.sort_index(ascending=True)

users_genres = users_genres.set_index('user_id')
users_genres = users_genres.sort_values(by="user_id", ascending=True)

display(users_genres)
display(users_df)


,genres
user_id,
1,Horror|Mystery|Thriller
1,Comedy|Fantasy|Romance
1,Horror|Thriller
1,Adventure|Children|Fantasy
1,Drama|Sci-Fi
...,...
138493,Adventure|Animation|Children|Fantasy|Musical
138493,Drama|Romance|War
138493,Action|Adventure


""
user_id
1
2
3
4
5
...
138489
138490
138491


In [26]:
def count_rated_genres(user_id, df):

    users_genre = df.loc[user_id,:]
    rated_genres = set()

    for genre in users_genre.genres:
        rated_genres.update(genre.split('|'))

    return len(rated_genres)

users_rated_genres = []
for user in users_df.index:
     users_rated_genres.append(count_rated_genres(user, users_genres))


In [27]:
users_rated_genres_df = pd.DataFrame({
    'user_id' : users_df.index.tolist(),
    'rated_genres' : users_rated_genres
})

display(users_rated_genres_df)


mean_rated_genres = np.mean(users_rated_genres)
print(mean_rated_genres)
users_rated_genres_df = users_rated_genres_df[users_rated_genres_df['rated_genres'] > mean_rated_genres]

display(users_rated_genres_df)


,user_id,rated_genres
0,1,17
1,2,18
2,3,18
3,4,15
4,5,17
...,...,...
138488,138489,17
138489,138490,18
138490,138491,14
138491,138492,17


16.379679839414266


,user_id,rated_genres
0,1,17
1,2,18
2,3,18
4,5,17
6,7,18
...,...,...
138486,138487,17
138488,138489,17
138489,138490,18
138491,138492,17


In [28]:
count_ratings = ratings_df.groupby(['user_id']).count()
selected = count_ratings['rating'] > count_ratings['rating'].mean()
expert_users = count_ratings.loc[selected]
expert_users = pd.DataFrame(expert_users)
expert_users = expert_users.reset_index()
expert_users = expert_users.merge(users_rated_genres_df, how='inner', left_on='user_id', right_on='user_id')
expert_users = expert_users.drop(['movie_id'], axis=1)
expert_users.rename(columns={'rating' : 'num_ratings'}, inplace=True)

display(expert_users)


,user_id,num_ratings,rated_genres
0,1,175,17
1,3,187,18
2,7,276,18
3,11,504,19
4,14,243,17
...,...,...,...
35895,138483,276,19
35896,138484,148,18
35897,138486,193,18
35898,138490,151,18


In [29]:
#users amount
original_users_amount = 0
for i in range(0, ratings_df.shape[0]):
    if i != 0:
        if ratings_df.user_id[i] != ratings_df.user_id[i-1]:
            original_users_amount += 1
    else:
        original_users_amount += 1
print(original_users_amount)
reduced_users_amount = expert_users.shape[0]
print(reduced_users_amount)

#ratings amount
original_ratings_amount = ratings_df.shape[0]
print(original_ratings_amount)
reduced_ratings_amount = 0
for i in range(0, expert_users.shape[0]):
    reduced_ratings_amount += expert_users.num_ratings[i]
print(reduced_ratings_amount)


138493
35900
20000263
13977680


In [14]:
#given user and movie --> predict rate
#kNN with average grade for each genre given by user

from sklearn.neighbors import KNeighborsRegressor

kNN = KNeighborsRegressor(n_neighbors=5)

#Load Data
data_folder = "C:/Users/roman/OneDrive/Bureau/UM/BA/Y3/P1/RS/Group Assignment/archive"
average_ratings_per_user_for_each_genre_df = pd.read_csv(data_folder + "/data.csv")
data_df = average_ratings_per_user_for_each_genre_df.fillna(0)
data_df = data_df.drop(8041)
display(data_df)

X = data_df.to_numpy()
Y = ratings_df.to_numpy()

kNN.fit(X, Y)

print(kNN.predict())

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,3.0,4.363636,4.000000,4.057692,3.750000,4.300000,4.666667,4.220000,4.333333,4.260000,...,4.062500,4.285714,0.0,0.000000,4.300000,3.000000,4.000000,4.000000,4.224138,3.937500
1,6.0,4.500000,1.000000,3.666667,5.000000,3.000000,0.000000,3.750000,0.000000,3.500000,...,3.888889,3.500000,0.0,0.000000,3.500000,0.000000,3.666667,0.000000,4.333333,0.000000
2,8.0,4.666667,3.600000,3.640000,3.714286,3.428571,4.800000,3.736842,5.000000,3.923077,...,3.952381,3.692308,0.0,3.666667,3.285714,0.000000,3.181818,0.000000,4.192308,4.333333
3,10.0,3.000000,4.000000,4.000000,4.000000,4.000000,3.818182,3.909091,3.000000,3.500000,...,3.900000,4.250000,0.0,0.000000,4.000000,0.000000,3.833333,0.000000,3.840000,4.500000
4,11.0,4.083333,3.900000,3.748276,4.090909,3.887324,3.892857,4.085890,3.500000,3.923977,...,4.093220,3.482759,0.0,4.574074,4.059524,4.750000,4.037118,4.142857,4.020134,3.814103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8036,22352.0,3.366279,3.631313,3.362840,3.676471,3.766667,3.724490,3.509479,3.558824,3.496324,...,3.456233,3.532895,0.0,3.776316,3.465517,2.875000,3.528302,3.636364,3.480826,3.644444
8037,22354.0,4.076923,4.333333,3.944444,4.500000,4.714286,4.684211,3.900000,4.500000,4.220588,...,4.018182,4.185185,0.0,0.000000,4.333333,4.454545,3.958333,4.375000,4.246667,4.055556
8038,22357.0,3.492537,3.776596,3.671587,3.780000,3.685185,3.703704,3.298507,2.818182,3.412281,...,3.610778,3.760714,0.0,4.500000,3.744898,4.033333,3.284091,3.710526,3.723553,3.390909
8039,22363.0,2.023810,1.265625,1.687097,1.526786,1.213675,2.272727,1.666667,1.500000,1.898936,...,1.701681,2.068627,0.0,1.944444,1.396104,3.500000,1.768116,1.000000,1.925325,1.720588


ValueError: Found input variables with inconsistent numbers of samples: [8041, 20000263]

20
['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical', 'Western', 'Film-Noir', '(no genres listed)']
